<a href="https://colab.research.google.com/github/driphics/GPT-2-model/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!git clone https://github.com/openai/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 209, done.
remote: Total 209 (delta 0), reused 0 (delta 0), pack-reused 209
Receiving objects: 100% (209/209), 4.37 MiB | 19.38 MiB/s, done.
Resolving deltas: 100% (109/109), done.


In [0]:
import os
os.chdir("gpt-2")

In [11]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 730kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 58.2Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 721kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:23, 59.6Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 6.68Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 53.1Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 45.6Mit/s]                                                       


In [0]:
!export PYTHONIOENCODING=UTF-8

In [13]:
os.chdir('src')
!pip install regex

     |████████████████████████████████| 655kB 2.8MB/s 
  Created wheel for regex: filename=regex-2019.8.19-cp36-cp36m-linux_x86_64.whl size=609236 sha256=9e3ca5b630cad5692392f7e383176b9a485b6958d1fab149ecc0f888311a098d
  Stored in directory: /root/.cache/pip/wheels/90/04/07/b5010fb816721eb3d6dd64ed5cc8111ca23f97fdab8619b5be
Successfully built regex


In [0]:
import json
import os
import numpy as np
import tensorflow as tf
import model, sample, encoder

def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [0]:
interact_model(
    '345M',
    None,
    1,
    1,
    300,
    1,
    0,
    '/content/gpt-2/models'
)